In [6]:
from processing import preprocessing,get_movies_aspect_matrix,dict_movie_aspect,viewed_matrix
import json
import pandas as pd
import numpy as np
import scipy
import sklearn.preprocessing as pp
from compute_strength import film_strength
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt

In [7]:
trainset = pd.read_csv('/home/jinfeng/Downloads/technique/dataset/context dataset/Mobile_Frappe/frappe/submission/recsys2023/data/train.csv')
testset = pd.read_csv('/home/jinfeng/Downloads/technique/dataset/context dataset/Mobile_Frappe/frappe/submission/recsys2023/data/test.csv')
trainset['cnt'] = (trainset['cnt'] + 1) * 2 + 1
testset['cnt'] = (testset['cnt'] + 1) * 2 + 1

In [8]:
max(trainset['cnt']),max(testset['cnt'])

(5.0, 4.6222575010097575)

In [4]:
meta_app_knowledge

NameError: name 'meta_app_knowledge' is not defined

In [ ]:
trainset

In [9]:
meta_app_knowledge = pd.read_csv('/home/jinfeng/Downloads/technique/dataset/context dataset/Mobile_Frappe/frappe/submission/recsys2023/data/meta_app.csv')

In [10]:
hehe_test = trainset.merge(meta_app_knowledge,on='item')
df_empty = testset.merge(meta_app_knowledge,on='item')

In [11]:
df_all = hehe_test.append(df_empty)

<ipython-input-11-e39932f4636d>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = hehe_test.append(df_empty)


In [12]:
df_all = df_all.rename(columns={'user':'userID','item':'itemID','cnt':'rating'})

In [13]:
df_sum = df_all.copy()
movie_set = set(df_sum['itemID'])
movie = df_sum[['itemID', 'category', 'downloads', 'language',
                'price', 'rating']].loc[df_sum[['itemID', 'category', 'downloads', 'language',
                                                'price', 'rating']].astype(str).drop_duplicates().index]
movie.index = range(len(movie))
movie = movie[['itemID', 'category', 'downloads', 'language',
               'price', 'rating']].loc[movie[['itemID', 'category', 'downloads', 'language',
                                              'price', 'rating']].astype(str).drop_duplicates().index]

In [14]:
ratings_dict = dict()
train_ratings_dict = dict()

train_ratings_dict["userID"] = []
train_ratings_dict["itemID"] = []
train_ratings_dict["rating"] = []


test_eva_dict = hehe_test.to_dict('records')
for row in test_eva_dict[:]:
    tuple_key = (row['user'],row['item'])
    ratings_dict[tuple_key] = row['cnt']
    train_ratings_dict["userID"].append(row['user'])
    train_ratings_dict["itemID"].append(row['item'])
    train_ratings_dict["rating"].append(row['cnt'])

In [15]:
compressed_test_ratings_dict = dict()
test_eva_dict = df_empty.to_dict('records')
for row in test_eva_dict[:]:
    if row['user'] not in compressed_test_ratings_dict:
        compressed_test_ratings_dict[row['user']] = []
    compressed_test_ratings_dict[row['user']].append((row['item'],row['cnt']))

In [16]:
df_all.columns


Index(['userID', 'itemID', 'rating', 'daytime', 'weekday', 'isweekend',
       'homework', 'weather', 'country', 'city', 'category', 'downloads',
       'language', 'price', 'rating'],
      dtype='object')

In [17]:
films = dict()


test_eva_dict = movie.to_dict('records')
for row in test_eva_dict[:]:
    if row['itemID'] not in films:
        films[row['itemID']] = dict()
    films[row['itemID']]['category'] = [row['category']]
    films[row['itemID']]['downloads'] = [row['downloads']]
    films[row['itemID']]['language'] = [row['language']]
    films[row['itemID']]['price'] = [row['price']]
    films[row['itemID']]['rating'] = [row['rating']]
    
    
    

<ipython-input-17-dcbc40360573>:4: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  test_eva_dict = movie.to_dict('records')


In [18]:
movies_watched = viewed_matrix(train_ratings_dict, films, 'netflix')
movies_watched = pd.DataFrame.from_dict(
    movies_watched, dtype='int64', orient='index').T
movies_watched = movies_watched.replace(np.nan, 0)

<ipython-input-18-7b9550262de4>:2: FutureWarning: In a future version, passing float-dtype values containing NaN and an integer dtype will raise IntCastingNaNError (subclass of ValueError) instead of silently ignoring the passed dtype. To retain the old behavior, call Series(arr) or DataFrame(arr) without passing a dtype.
  movies_watched = pd.DataFrame.from_dict(


In [19]:
user_ids_in_matrix = movies_watched.index.values
movies_watched = movies_watched.T
movies_watched = scipy.sparse.csc_matrix(movies_watched.values)
normalized_matrix_by_column = pp.normalize(
    movies_watched.tocsc(), norm='l2', axis=0)
cosine_sims = normalized_matrix_by_column.T * normalized_matrix_by_column
assert cosine_sims.shape[0] == cosine_sims.shape[1] == len(
    user_ids_in_matrix)

In [20]:
sims = dict()
for i in user_ids_in_matrix:
    sims[i] = []
cosine_sims = cosine_sims.todok().items()

for ((row, col), sim) in cosine_sims:
    if row != col:
        sims[user_ids_in_matrix[row]].append(
                (user_ids_in_matrix[col], sim))

In [21]:
category_in_db, movies_all_category_matrix = get_movies_aspect_matrix(
    films, "category")
downloads_in_db, movies_all_downloads_matrix = get_movies_aspect_matrix(
    films, "downloads")
language_in_db, movies_all_language_matrix = get_movies_aspect_matrix(
    films, "language")
price_in_db, movies_all_price_matrix = get_movies_aspect_matrix(
    films, "price")
rating_in_db, movies_all_rating_matrix = get_movies_aspect_matrix(
    films, "rating")


We have 32 category (an example is 31)


/home/jinfeng/Downloads/technique/dataset/context dataset/Mobile_Frappe/frappe/submission/recsys2023/A-I/processing.py:73: FutureWarning: In a future version, passing float-dtype values containing NaN and an integer dtype will raise IntCastingNaNError (subclass of ValueError) instead of silently ignoring the passed dtype. To retain the old behavior, call Series(arr) or DataFrame(arr) without passing a dtype.
  movies_all_aspects_matrix = pd.DataFrame.from_dict(


We have 16 downloads (an example is 47)
We have 28 language (an example is 74)
We have 3 price (an example is 78)
We have 4 rating (an example is 82)


In [22]:
df_all.columns


Index(['userID', 'itemID', 'rating', 'daytime', 'weekday', 'isweekend',
       'homework', 'weather', 'country', 'city', 'category', 'downloads',
       'language', 'price', 'rating'],
      dtype='object')

In [23]:
MUR = 0.8
MUC = 0.1
MUD = 0.1
MUL = 0.1
MUP = 0.1
MURR= 0.1

In [ ]:
predictions = []
for user_id, true_ratings in compressed_test_ratings_dict.items():
    if true_ratings:
        for (film_id, str_rating) in true_ratings:
            str_rating = str_rating * 4.458668059764898 / 5
            strength = film_strength(MUR, MUC, MUD, MUL, MUP, MURR, user_id, film_id, films, ratings_dict,
                                     sims[user_id], movies_all_category_matrix, movies_all_downloads_matrix, movies_all_language_matrix,movies_all_price_matrix,movies_all_rating_matrix)
            strength = strength * 4.458668059764898 / 5
            predictions.append((float(str_rating), float(strength)))



In [24]:
true_ratings = [x for (x, y) in predictions]
predicted_ratings = [round(y) for (x, y) in predictions]
sqrt(mean_squared_error(true_ratings, predicted_ratings)), mean_absolute_error(true_ratings, predicted_ratings)

(1.1711929765078264, 0.9848455142451471)

In [28]:
hehe_test

,user,item,cnt,daytime,weekday,isweekend,homework,weather,country,city,category,downloads,language,price,rating
0,12,2470,0.000000,4,10,14,16,19,97,100,31,47,74,78,82
1,12,2470,1.350314,4,13,15,16,19,97,100,31,47,74,78,82
2,12,2470,0.000000,6,11,15,16,25,97,100,31,47,74,78,82
3,173,25,1.012735,6,8,15,17,25,97,257,8,36,56,76,81
4,324,25,0.000000,1,11,15,17,26,79,237,8,36,56,76,81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75768,436,3821,0.337578,1,7,14,17,19,78,233,10,37,56,76,81
75769,121,3634,1.167829,0,9,14,16,25,97,100,31,47,74,78,82
75770,92,1291,1.407677,3,9,14,17,25,73,100,27,34,56,76,80
75771,293,3875,0.337578,0,9,14,17,26,99,100,31,47,74,78,82


In [29]:
user_item = {}
user_item_neg = {}

for user in set(df_all['userID']):
    if user not in user_item:
        pos_df = hehe_test[hehe_test['user'] == user]
        user_item[user] = set(pos_df['item'].values)
for user,pos_item_set in user_item.items():
    unwatched_set = movie_set - pos_item_set
    if user not in user_item_neg:
        user_item_neg[user] = set()
    user_item_neg[user] = set(np.random.choice(list(unwatched_set), size=100, replace=False))
    

In [30]:
num_i = 101


def get_prediction_list(user, target, k):

    item_neg = user_item_neg[user]
    test_ratings_dict = dict()
    test_ratings_dict[user] = []
    items_list = []
    for item in item_neg:
        test_ratings_dict[user].append((item, 1))
        items_list.append(item)
    items_list.append(target)
    test_ratings_dict[user].append((target, 1))
    predictions = []
    for user_id, true_ratings in test_ratings_dict.items():
        if true_ratings:
            for (film_id, str_rating) in true_ratings:
                strength = film_strength(MUR, MUC, MUD, MUL, MUP, MURR, user_id, film_id, films, ratings_dict,
                                         sims[user_id], movies_all_category_matrix, movies_all_downloads_matrix, movies_all_language_matrix, movies_all_price_matrix, movies_all_rating_matrix)
                predictions.append((int(str_rating), strength))

    predicted_ratings = [round(y) for (x, y) in predictions]

    prediction_dict = dict(zip(items_list, predicted_ratings))
    top_k = dic_order_value_and_get_key(prediction_dict, k)
    return top_k, prediction_dict

In [33]:
import math
from tqdm import tqdm
def hr_ndcg():
    hr1 = 0.0
    ndcg1 = 0.0
    hr2 = 0.0
    ndcg2 = 0.0
    test_eva_dict = df_empty.to_dict('records')
    for row in tqdm(test_eva_dict[:]):
        user = row['user']
        target = row['item']
        
        
        recommended_item_10,_ = get_prediction_list(user,target,10)
        recommended_item_20,_ = get_prediction_list(user,target,20)
        if target in recommended_item_10:
            hr1 +=1
            posi = recommended_item_10.index(target)
            ndcg1 += 1 / math.log(posi + 2,2)
        if target in recommended_item_20:
            hr2 +=1
            posi = recommended_item_20.index(target)
            ndcg2 += 1 / math.log(posi + 2,2)
    return hr1 / df_empty.shape[0], ndcg1 / df_empty.shape[0],hr2 / df_empty.shape[0], ndcg2 / df_empty.shape[0]
        

In [35]:
def dic_order_value_and_get_key(dicts, count):
    final_result = []
    a = sorted(dicts.items(), key=lambda x: x[1], reverse=True)
    a_dict = a[:count]
    for l in a_dict:
        final_result.append(l[0])
    return final_result

In [36]:
hr_ndcg()

100%|██████████| 18943/18943 [42:28:16<00:00,  8.07s/it]   


(0.001108588924668743,
 0.0004425044595498827,
 0.020429710183181123,
 0.005327836664330952)

In [48]:
movie_rank = movie.copy()
movie_rank = movie_rank.iloc[:, :-2]
movie_rank = pd.concat([movie_rank,movie.iloc[:,-1]],axis=1)

In [300]:
import torch
import pandas as pd
import numpy as np
from torch import nn
import torch
from torch import nn
from torch.nn import LeakyReLU
leaky = LeakyReLU(0.1)
drop = torch.nn.Dropout(p=0.4)
class aggregator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(aggregator, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        #self.wei = torch.nn.Embedding(self.input_dim,self.output_dim)
        #self.bias = torch.nn.Embedding(1,self.output_dim)

    def forward(self, user, to_agg, rel):
        '''
        user: 128 * 64
        rel: 64 * 5
        to_agg: 128 * 5 * 64
        '''
        scores = torch.matmul(user, rel) # 128 * 5
        scores = leaky(scores)
        m = torch.nn.Softmax(dim=1) # 128 * 5
        scores = m(scores) # 128 * 5
        scores1 = scores.unsqueeze(1) #128 * 1 * 5
        context_agg = torch.bmm(scores1, to_agg) # 128 * 1 * 64
        return context_agg



class context_aware_knowledge(nn.Module):
    def __init__(self, n_users, n_contexts, n_rc, n_entity, n_kc, n_factors,context_or):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, n_factors)
        self.context_factors = torch.nn.Embedding(n_contexts, n_factors)
        self.relation_c = torch.nn.Embedding(n_factors, n_rc)
        self.entity_factors = torch.nn.Embedding(n_entity, n_factors)
        self.relation_k = torch.nn.Embedding(n_factors, n_kc) # 64 * 5
        self.agg = aggregator(n_factors, n_factors)
        self.context_or = context_or

    def forward(self, user, contexts_index, entities_index):

        entities = self.entity_factors(entities_index)  # 128 * 5 * 64
        contexts = self.context_factors(contexts_index)  # 128 * 5 * 64
        if self.context_or == True:
            u_nei = self.agg(self.user_factors(user), contexts,
                    self.relation_c.weight)  # 128 * 1 * 64
            u_final = u_nei + self.user_factors(user).unsqueeze(1) # 128 * 1 * 64
            u_final = leaky(u_final)
        else:
            u_final = self.user_factors(user)
        importances = torch.matmul(u_final.squeeze(1), self.relation_k.weight) # 128 * 5
        importances = leaky(importances)
        m = torch.nn.Softmax(dim=1) # 128 * 5
        importances = m(importances) # 128 * 5
        scores = torch.bmm(entities,u_final.squeeze(1).unsqueeze(2)) # 128 * 5 * 1
        scores_final = torch.bmm(importances.unsqueeze(1),scores) # 128 * 1 * 1
        scores_final = scores_final.squeeze(2) # 128 * 1
        scores_final = scores_final.squeeze(1) # 128,
#         scores_final = scores.sum(1) / 5
#         scores_final = scores_final.squeeze(1)
        return scores_final,importances,scores 

In [301]:
path = '/home/jinfeng/Downloads/technique/dataset/context dataset/Mobile_Frappe/frappe/submission/recsys2023/models/abalation/model_nocontextaverage0.005_256_5e-05_256.pkl'

In [302]:
meta_app_knowledge = pd.read_csv('/home/jinfeng/Downloads/technique/dataset/context dataset/Mobile_Frappe/frappe/submission/recsys2023/data/meta_app.csv')
n_entities = max(meta_app_knowledge['rating']) + 1
n_rel = meta_app_knowledge.shape[1] - 1
df = pd.read_csv('/home/jinfeng/Downloads/technique/dataset/context dataset/Mobile_Frappe/frappe/submission/recsys2023/data/final_app.csv')
n_cf = df.shape[1] - 3  # Here, we compute the number of contextual factors
# Here, we compute the number of users
n_users = len(df['user'].value_counts())
# Here, we compute the number of items
n_items = len(df['item'].value_counts())
n_contexts = max(df['city']) + 1

df_test = pd.read_csv('/home/jinfeng/Downloads/technique/dataset/context dataset/Mobile_Frappe/frappe/submission/recsys2023/data/test.csv')

In [303]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = context_aware_knowledge(n_users, n_contexts, n_cf, n_entities, n_rel,256,True).to(DEVICE)
load_params = torch.load(path)['model']
model_params = model.state_dict()
same_parsms = {k: v for k, v in load_params.items()
               if k in model_params.keys()}
model_params.update(same_parsms)

model.load_state_dict(model_params)

<All keys matched successfully>

In [317]:
import pickle
f_save = open('to_user.pkl', 'wb')
pickle.dump(user_item_neg, f_save)
f_save.close()
 
# # # 读取
f_read = open('to_user.pkl', 'rb')
user_item_neg = pickle.load(f_read)
#print(dict2)



In [241]:
user_item = {}
user_item_neg = {}

for user in set(df_all['userID']):
    if user not in user_item:
        pos_df = hehe_test[hehe_test['user'] == user]
        user_item[user] = set(pos_df['item'].values)
for user,pos_item_set in user_item.items():
    unwatched_set = movie_set - pos_item_set
    if user not in user_item_neg:
        user_item_neg[user] = set()
    user_item_neg[user] = set(np.random.choice(list(unwatched_set), size=100, replace=False))
    

In [272]:
user_item_neg = {}
i = 0
test_eva_dict = df_empty.to_dict('records')
for row in tqdm(test_eva_dict[:]):
    user_item_neg[(i,row['user'])] = set(np.random.choice(list(movie_set-set([row['item']])), size=100, replace=False))
    i = i + 1

100%|██████████| 18943/18943 [00:06<00:00, 3035.32it/s]


In [306]:
num_i = 101
def get_prediction_list(model,i,user, target, contexts_index,entities_index,k):
    item_neg = user_item_neg[(i,user)]
    movie_rec = meta_app_knowledge[meta_app_knowledge['item'].isin(item_neg)]
    entities_o = movie_rec.iloc[:, 1:].values
    entities = np.insert(entities_o,0,values=entities_index,axis=0)
    
    items_o = movie_rec['item'].values
    items = np.insert(items_o,0,np.array(target))
    
    user = np.array([user] * num_i)
    user = torch.LongTensor(user).to(DEVICE)
    entities = torch.LongTensor(entities).to(DEVICE)
    contexts = np.tile(contexts_index,(num_i,1))
    contexts = torch.LongTensor(contexts).to(DEVICE)
    prediction,importances,scores = model(user, contexts, entities)
    prediction = prediction.detach().numpy()
    prediction_dict = dict(zip(items, prediction))
    top_k = dic_order_value_and_get_key(prediction_dict,k)
    return top_k,prediction_dict

In [309]:
i = 0
for row in tqdm(test_eva_dict[:1]):
    user = row['user']
    target = row['item']
    contexts_index = []
    contexts_index.append(row['daytime'])
    contexts_index.append(row['weekday'])
    contexts_index.append(row['isweekend'])
    contexts_index.append(row['homework'])
    contexts_index.append(row['weather'])
    contexts_index.append(row['country'])
    contexts_index.append(row['city'])
    contexts_index = np.array(contexts_index)
    entities_index = []

    entities_index.append(row['category'])
    entities_index.append(row['downloads'])
    entities_index.append(row['language'])
    entities_index.append(row['price'])
    entities_index.append(row['rating'])
    entities_index = np.array(entities_index)

100%|██████████| 1/1 [00:00<00:00, 4739.33it/s]


In [310]:
recommended_item_10, prediction_dict_10= get_prediction_list(
    model, i, user, target, contexts_index, entities_index, 10)
recommended_item_20, prediction_dict_20 = get_prediction_list(
    model, i, user, target, contexts_index, entities_index, 20)

In [278]:
import math
from tqdm import tqdm
def hr_ndcg(data,model):
    hr1 = 0.0
    ndcg1 = 0.0
    hr2 = 0.0
    ndcg2 = 0.0
    test_eva_dict = df_empty.to_dict('records')
    i = 0
    for row in tqdm(test_eva_dict[:]):
        user = row['user']
        target = row['item']
        contexts_index = []
        contexts_index.append(row['daytime'])
        contexts_index.append(row['weekday'])
        contexts_index.append(row['isweekend'])
        contexts_index.append(row['homework'])
        contexts_index.append(row['weather'])
        contexts_index.append(row['country'])
        contexts_index.append(row['city'])
        contexts_index = np.array(contexts_index)
        entities_index = []
        
        entities_index.append(row['category'])
        entities_index.append(row['downloads'])
        entities_index.append(row['language'])
        entities_index.append(row['price'])
        entities_index.append(row['rating'])
        entities_index = np.array(entities_index)
        
        
        recommended_item_10,_ = get_prediction_list(model,i,user,target,contexts_index,entities_index,10)
        recommended_item_20,_ = get_prediction_list(model,i,user,target,contexts_index,entities_index,20)
        if target in recommended_item_10:
            hr1 +=1
            posi = recommended_item_10.index(target)
            ndcg1 += 1 / math.log(posi + 2,2)
        if target in recommended_item_20:
            hr2 +=1
            posi = recommended_item_20.index(target)
            ndcg2 += 1 / math.log(posi + 2,2)
        i = i + 1
    return hr1 / data.shape[0], ndcg1 / data.shape[0],hr2 / data.shape[0], ndcg2 / data.shape[0]
        

In [304]:
path = '/home/jinfeng/Downloads/technique/dataset/context dataset/Mobile_Frappe/frappe/submission/recsys2023/models/model_0.005_256_5e-05_256.pkl'
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = context_aware_knowledge(n_users, n_contexts, n_cf, n_entities, n_rel,256,True).to(DEVICE)
load_params = torch.load(path)['model']
model_params = model.state_dict()
same_parsms = {k: v for k, v in load_params.items()
               if k in model_params.keys()}
model_params.update(same_parsms)

model.load_state_dict(model_params)
#hr_ndcg(df_empty,model)

<All keys matched successfully>

In [248]:
(0.26648366151084835,
 0.14949698722591728,
 0.3940241777965475,
 0.1815652247647183)

(0.26648366151084835,
 0.14949698722591728,
 0.3940241777965475,
 0.1815652247647183)

In [280]:
num_i = 101
def get_prediction_list(model,i,user, target,contexts_index,k):
    item_neg = user_item_neg[(i,user)]
    items = list(item_neg)
    items.append(target)
    #movie_rec = meta_app_knowledge[meta_app_knowledge['item'].isin(item_neg)]
    #movie_rec.index = range(1, len(movie_rec)+1)
    #movie_rec.loc[0] = entities_index
    user = np.array([user] * num_i)
    user = torch.LongTensor(user).to(DEVICE)
    #item = np.array([target] * num_i)
    item = torch.LongTensor(items).to(DEVICE)
    #entities = movie_rec.iloc[:, 1:].values
    #entities = torch.LongTensor(entities).to(DEVICE)
    contexts = np.tile(contexts_index,(num_i,1))
    contexts = torch.LongTensor(contexts).to(DEVICE)
    prediction = model(user, item, contexts)
    prediction = prediction.detach().numpy()
    prediction_dict = dict(zip(items, prediction))
    top_k = dic_order_value_and_get_key(prediction_dict,k)
    return top_k,prediction_dict

In [281]:
import math
def hr_ndcg(data,model):
    hr1 = 0.0
    ndcg1 = 0.0
    hr2 = 0.0
    ndcg2 = 0.0
    test_eva_dict = df_empty.to_dict('records')
    i = 0
    for row in test_eva_dict[:]:
        user = row['user']
        target = row['item']
        contexts_index = []
        contexts_index.append(row['daytime'])
        contexts_index.append(row['weekday'])
        contexts_index.append(row['isweekend'])
        contexts_index.append(row['homework'])
        contexts_index.append(row['weather'])
        contexts_index.append(row['country'])
        contexts_index.append(row['city'])
        contexts_index = np.array(contexts_index)
#         entities_index = []
#         entities_index.append(target)
#         entities_index.append(row['category'])
#         entities_index.append(row['downloads'])
#         entities_index.append(row['language'])
#         entities_index.append(row['price'])
#         entities_index.append(row['rating'])
#         entities_index = np.array(entities_index)
        
        
        recommended_item_10,_ = get_prediction_list(model,i,user,target,contexts_index,10)
        recommended_item_20,_ = get_prediction_list(model,i,user,target,contexts_index,20)
        if target in recommended_item_10:
            hr1 +=1
            posi = recommended_item_10.index(target)
            ndcg1 += 1 / math.log(posi + 2,2)
        if target in recommended_item_20:
            hr2 +=1
            posi = recommended_item_20.index(target)
            ndcg2 += 1 / math.log(posi + 2,2)
        i = i + 1
    return hr1 / data.shape[0], ndcg1 / data.shape[0],hr2 / data.shape[0], ndcg2 / data.shape[0]
        

In [282]:
class NeuMF(nn.Module):
    def __init__(self, n_users, n_items, n_contexts,mlp_factors,gmf_factors, layers):
        super(NeuMF, self).__init__()
        self.n_users = n_users
        self.n_items = n_items
        self.mlp_factors = mlp_factors
        self.gmf_factors = gmf_factors
        self.layers = layers
        
        self.mlp_user_factors = torch.nn.Embedding(n_users, mlp_factors)
        self.mlp_item_factors = torch.nn.Embedding(n_items, mlp_factors)
        self.mlp_context_factors = torch.nn.Embedding(n_contexts, mlp_factors)
        
        self.gmf_user_factors = torch.nn.Embedding(n_users, gmf_factors)
        self.gmf_item_factors = torch.nn.Embedding(n_items, gmf_factors)
        
        self.MLP1 = nn.Sequential(nn.Linear(9 * mlp_factors, self.layers[0]))
        self.MLP2 = nn.Sequential(nn.Linear(self.layers[0],self.layers[1]))
        self.MLP3 = nn.Sequential(nn.Linear(self.layers[1],self.layers[2]))
        #self.MLP4 = nn.Sequential(nn.Linear(self.layers[2],self.layers[3]))
        
        self.outlayer = nn.Sequential(nn.Linear(self.layers[2] + gmf_factors,1))
        
        
    def forward(self, user_id, item_id,context_id):
        GMF_user_embeddings = self.gmf_user_factors(user_id) ## 128 * 64
        GMF_item_embeddings = self.gmf_item_factors(item_id) ## 128 * 64
        GMF_merge_embedding = GMF_user_embeddings * GMF_item_embeddings ## 128 * 64
        
        
        MLP_user_embeddings = self.mlp_user_factors(user_id) ## 128 * 64
        MLP_item_embeddings = self.mlp_item_factors(item_id) ## 128 * 64
        MLP_context_embeddings = self.mlp_user_factors(context_id)
        MLP_merge_embedding = torch.cat((MLP_user_embeddings,MLP_item_embeddings),1) ## 128 * 128
        MLP_context_embedding = torch.reshape(MLP_context_embeddings,(user_id.shape[0],-1))
        MLP_merge_embedding = torch.concat((MLP_merge_embedding,MLP_context_embedding),1)
        
        
        MLP_out1 = self.MLP1(MLP_merge_embedding) ## 128 * 64
        MLP_out1 = torch.nn.ReLU()(MLP_out1)
        
        MLP_out2 = self.MLP2(MLP_out1) ## 128 * 32
        MLP_out2 = torch.nn.ReLU()(MLP_out2)
        
        MLP_out3 = self.MLP3(MLP_out2) ## 128 * 16
        MLP_out3 = torch.nn.ReLU()(MLP_out3)
        
        #MLP_out4 = self.MLP4(MLP_out3) ## 128 * 8
        #MLP_out4 = torch.nn.ReLU()(MLP_out4)
        
        out = torch.cat((GMF_merge_embedding,MLP_out3),1)
        
        
        out = self.outlayer(out)  ## 128 * 1
        return out.squeeze(1)   


In [283]:
# def RMSE(data, model):
#     users_index = data.iloc[:, 0].values
#     users = torch.LongTensor(users_index).to(DEVICE)
#     items_index = data.iloc[:, 1].values
#     items = torch.LongTensor(items_index).to(DEVICE)
#     context_index = data.iloc[:, 3:].values
#     contexts = torch.LongTensor(context_index).to(DEVICE)
    
    
#     rating = torch.FloatTensor(data.iloc[:, 2].values).to(DEVICE)
#     prediction= model(users, items,contexts)
#     rmse = loss_func(prediction, rating)
#     mae = torch.nn.L1Loss()(prediction, rating)
#     return rmse ** 0.5,mae
# path = '/home/jinfeng/Downloads/technique/dataset/context dataset/Mobile_Frappe/frappe/submission/recsys2023/models/model_neumf_contexts.pkl'
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# model = NeuMF(n_users, n_items,n_contexts, mlp_factors,gmf_factors, layers).to(DEVICE)
# load_params = torch.load(path)['model']
# model_params = model.state_dict()
# same_parsms = {k: v for k, v in load_params.items()
#                if k in model_params.keys()}
# model_params.update(same_parsms)
# model.load_state_dict(model_params)
# RMSE(df_empty,model)

In [284]:
max(df_empty['cnt'])

4.6222575010097575

In [285]:
mlp_factors = 8
gmf_factors = 32 
layers = [16,8,4]
path = '/home/jinfeng/Downloads/technique/dataset/context dataset/Mobile_Frappe/frappe/submission/recsys2023/models/model_neumf_contexts.pkl'
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = NeuMF(n_users, n_items,n_contexts, mlp_factors,gmf_factors, layers).to(DEVICE)
load_params = torch.load(path)['model']
model_params = model.state_dict()
same_parsms = {k: v for k, v in load_params.items()
               if k in model_params.keys()}
model_params.update(same_parsms)

model.load_state_dict(model_params)

hr_ndcg(df_test,model)

(0.4054795966847912,
 0.20809070671672492,
 0.5722430449242464,
 0.25035328556042724)

In [286]:
class NeuMF(nn.Module):
    def __init__(self, n_users, n_items, mlp_factors,gmf_factors, layers):
        super(NeuMF, self).__init__()
        self.n_users = n_users
        self.n_items = n_items
        self.mlp_factors = mlp_factors
        self.gmf_factors = gmf_factors
        self.layers = layers
        
        self.mlp_user_factors = torch.nn.Embedding(n_users, mlp_factors)
        self.mlp_item_factors = torch.nn.Embedding(n_items, mlp_factors)
        
        self.gmf_user_factors = torch.nn.Embedding(n_users, gmf_factors)
        self.gmf_item_factors = torch.nn.Embedding(n_items, gmf_factors)
        
        self.MLP1 = nn.Sequential(nn.Linear(2 * mlp_factors, self.layers[0]))
        self.MLP2 = nn.Sequential(nn.Linear(self.layers[0],self.layers[1]))
        self.MLP3 = nn.Sequential(nn.Linear(self.layers[1],self.layers[2]))
        #self.MLP4 = nn.Sequential(nn.Linear(self.layers[2],self.layers[3]))
        
        self.outlayer = nn.Sequential(nn.Linear(self.layers[2] + gmf_factors,1))
        
        
    def forward(self, user_id, item_id):
        GMF_user_embeddings = self.gmf_user_factors(user_id) ## 128 * 64
        GMF_item_embeddings = self.gmf_item_factors(item_id) ## 128 * 64
        GMF_merge_embedding = GMF_user_embeddings * GMF_item_embeddings ## 128 * 64
        
        
        MLP_user_embeddings = self.mlp_user_factors(user_id) ## 128 * 64
        MLP_item_embeddings = self.mlp_item_factors(item_id) ## 128 * 64
        MLP_merge_embedding = torch.cat((MLP_user_embeddings,MLP_item_embeddings),1) ## 128 * 128
        
        
        MLP_out1 = self.MLP1(MLP_merge_embedding) ## 128 * 64
        MLP_out1 = torch.nn.ReLU()(MLP_out1)
        
        MLP_out2 = self.MLP2(MLP_out1) ## 128 * 32
        MLP_out2 = torch.nn.ReLU()(MLP_out2)
        
        MLP_out3 = self.MLP3(MLP_out2) ## 128 * 16
        MLP_out3 = torch.nn.ReLU()(MLP_out3)
        
        #MLP_out4 = self.MLP4(MLP_out3) ## 128 * 8
        #MLP_out4 = torch.nn.ReLU()(MLP_out4)
        
        out = torch.cat((GMF_merge_embedding,MLP_out3),1)
        
        
        out = self.outlayer(out)  ## 128 * 1
        return out.squeeze(1)   


In [287]:
num_i = 101
def get_prediction_list(model,i,user, target,k):
    item_neg = user_item_neg[(i,user)]
    items = list(item_neg)
    items.append(target)
    #movie_rec = meta_app_knowledge[meta_app_knowledge['item'].isin(item_neg)]
    #movie_rec.index = range(1, len(movie_rec)+1)
    #movie_rec.loc[0] = entities_index
    user = np.array([user] * num_i)
    user = torch.LongTensor(user).to(DEVICE)
    #item = np.array([target] * num_i)
    item = torch.LongTensor(items).to(DEVICE)
    #entities = movie_rec.iloc[:, 1:].values
    #entities = torch.LongTensor(entities).to(DEVICE)
    #contexts = np.tile(contexts_index,(num_i,1))
    #contexts = torch.LongTensor(contexts).to(DEVICE)
    prediction = model(user, item)
    prediction = prediction.detach().numpy()
    prediction_dict = dict(zip(items, prediction))
    top_k = dic_order_value_and_get_key(prediction_dict,k)
    return top_k,prediction_dict

In [288]:
import math
def hr_ndcg(data,model):
    hr1 = 0.0
    ndcg1 = 0.0
    hr2 = 0.0
    ndcg2 = 0.0
    test_eva_dict = df_empty.to_dict('records')
    i = 0.0
    for row in tqdm(test_eva_dict[:]):
        user = row['user']
        target = row['item']
#         contexts_index = []
#         contexts_index.append(row['daytime'])
#         contexts_index.append(row['weekday'])
#         contexts_index.append(row['isweekend'])
#         contexts_index.append(row['homework'])
#         contexts_index.append(row['weather'])
#         contexts_index.append(row['country'])
#         contexts_index.append(row['city'])
#         contexts_index = np.array(contexts_index)
#         entities_index = []
#         entities_index.append(target)
#         entities_index.append(row['category'])
#         entities_index.append(row['downloads'])
#         entities_index.append(row['language'])
#         entities_index.append(row['price'])
#         entities_index.append(row['rating'])
#         entities_index = np.array(entities_index)
        
        
        recommended_item_10,_ = get_prediction_list(model,i,user,target,10)
        recommended_item_20,_ = get_prediction_list(model,i,user,target,20)
        if target in recommended_item_10:
            hr1 +=1
            posi = recommended_item_10.index(target)
            ndcg1 += 1 / math.log(posi + 2,2)
        if target in recommended_item_20:
            hr2 +=1
            posi = recommended_item_20.index(target)
            ndcg2 += 1 / math.log(posi + 2,2)
        i = i + 1
    return hr1 / data.shape[0], ndcg1 / data.shape[0],hr2 / data.shape[0], ndcg2 / data.shape[0]
        

In [289]:
path = '/home/jinfeng/Downloads/technique/dataset/context dataset/Mobile_Frappe/frappe/submission/recsys2023/models/model_neumf.pkl'
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = NeuMF(n_users, n_items, mlp_factors,gmf_factors, layers).to(DEVICE)
load_params = torch.load(path)['model']
model_params = model.state_dict()
same_parsms = {k: v for k, v in load_params.items()
               if k in model_params.keys()}
model_params.update(same_parsms)

model.load_state_dict(model_params)
hr_ndcg(df_test,model)

100%|██████████| 18943/18943 [00:12<00:00, 1477.26it/s]


(0.5273715884495592,
 0.2867889354701134,
 0.6723327878371957,
 0.32370759217578143)

In [290]:
class MF(nn.Module):
    def __init__(self, n_users, n_items,n_factors):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, n_factors)
        self.item_factors = torch.nn.Embedding(n_items, n_factors)
    def forward(self, user, item):
        u_final = self.user_factors(user).unsqueeze(1)
        i_final = self.item_factors(item).unsqueeze(1)
        u_final = drop(u_final)
        i_final = drop(i_final)
        return (u_final * i_final).sum(2).squeeze(1)

In [292]:
path = '/home/jinfeng/Downloads/technique/dataset/context dataset/Mobile_Frappe/frappe/submission/recsys2023/models/model_mf.pkl'
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = MF(n_users, n_items,256).to(DEVICE)
load_params = torch.load(path)['model']
model_params = model.state_dict()
same_parsms = {k: v for k, v in load_params.items()
               if k in model_params.keys()}
model_params.update(same_parsms)

model.load_state_dict(model_params)

hr_ndcg(df_test,model)

100%|██████████| 18943/18943 [00:11<00:00, 1705.29it/s]


(0.4317162012352848,
 0.19028969251812042,
 0.6723327878371957,
 0.2510244252670824)

In [94]:
df_test

,user,item,cnt,daytime,weekday,isweekend,homework,weather,country,city
0,356,314,-1.500000,2,9,14,17,19,97,100
1,25,1115,-0.369038,1,12,15,17,25,80,100
2,90,102,-1.500000,1,10,14,17,21,38,100
3,264,282,-0.894897,0,9,14,17,21,97,100
4,33,264,-1.500000,4,8,15,17,19,44,146
...,...,...,...,...,...,...,...,...,...,...
18938,107,319,-0.589835,6,7,14,17,19,48,139
18939,209,49,0.141393,2,11,15,17,25,97,100
18940,88,844,-1.500000,5,8,15,17,19,36,125
18941,95,550,-1.500000,1,10,14,16,25,36,100
